In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.models import Model
import glob
from keras.models import Sequential
from keras.layers import *
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks
remote = callbacks.RemoteMonitor(root='http://localhost:9000')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from tqdm import tqdm_notebook as tq
from random import shuffle
import os
from keras.utils.visualize_util import plot
from keras.optimizers import *
from shutil import copyfile

%matplotlib inline


Using Theano backend.
Using gpu device 0: GeForce GTX 970 (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5105)


We need to move boats around.

In [2]:
def group_boats(groups, new_folder = "./boats_id_train/"):
    files_list = glob.glob('./boats_id/*/*/*.jpg')
    tuples = [x.split("\\") for x in files_list]
    if not os.path.exists(new_folder):
        os.makedirs(new_folder)
        
    for i in range(len(groups) + 1):
        if not os.path.exists(new_folder + str(i) + "/"):
            os.makedirs(new_folder + str(i) + "/")
    
    moved = [False for _ in tuples]
    
    for i, group in enumerate(tq(groups)):
        if type(group) is int:
            group  = [group]
        
        for folder_idx in group:
            for j, tup in enumerate(tuples):
                if int(tup[2]) == folder_idx:
                    copyfile(files_list[j], new_folder + str(i + 1) + "/" + tup[-1])
                    moved[j] = True
    
    for j, move in enumerate(tq(moved)):
        if not move:
            copyfile(files_list[j], new_folder + "0" + "/" + tuples[j][-1])
    

In [23]:
group_boats([[0,1],[2,3],4,5,6,[7,8],[9,10],11,12,13,[14,15],20,22,36,26,27,32,42])

Now we need to have the same number of pictures in each class. So we'll add new ones.

In [3]:
def split_train_test(nb_images, train_dir):
    temp = train_dir.find("train")
    test_dir = train_dir[:temp] + "test" + train_dir[temp+5:]
    
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)
        
    classes = os.listdir(train_dir)
        
    train_dir = [train_dir + x + "/" for x in classes]
    test_dir = [test_dir + x + "/" for x in classes]
    
    for folder in test_dir:
        if not os.path.exists(folder):
            os.makedirs(folder)
    
    for train_class, test_class in zip(train_dir, test_dir):
        files = os.listdir(train_class)[:nb_images]
        for file in files:
            copyfile(train_class + file,test_class + file)
            os.remove(train_class + file)

In [5]:
split_train_test(60,"train/")

In [4]:
def create_new_pictures(train_folder, class_folder, generator,total_images, 
                        batch_size = 32, target_size = (256,256)):
    #count = len(os.listdir(train_folder + "/" + class_folder))
    count = 0  
    gen_folder = train_folder + "_gen/"
        
    if not os.path.exists(gen_folder):
        os.makedirs(gen_folder)
        
    if not os.path.exists(gen_folder + class_folder):
        os.makedirs(gen_folder + class_folder)
        
    
    progbar = tq(total = total_images, leave=False)
    progbar.update(count)
    
    
    for X in generator.flow_from_directory(train_folder, target_size=target_size,
                                             classes=[class_folder], 
                                             save_to_dir=train_folder + "_gen/" + class_folder, 
                                             batch_size=batch_size, color_mode="rgb"):
        count +=batch_size
        progbar.update(batch_size)
        if count >= total_images:
            break
    progbar.close()
    

In [5]:
my_generator = ImageDataGenerator(horizontal_flip=True,
    vertical_flip=True)

In [6]:
my_test_generator = ImageDataGenerator()

In [7]:
create_new_pictures("binary_dataset +imagenet + full frames/train", "fish", my_generator,100000, 8, (256,256))

Found 20809 images belonging to 1 classes.


KeyboardInterrupt: 

In [7]:
create_new_pictures("binary_dataset +imagenet + full frames/train", "fish", my_generator,20000, 8, (None,None))

Found 20810 images belonging to 1 classes.


KeyboardInterrupt: 

In [ ]:
for string in os.listdir("boats_id_train"):
    create_new_pictures("boats_id_train", string, my_generator,500, 32, (128,128))

In [10]:
def equilibrate_dataset(folder, nb_images_per_class, generator):
    c = 0
    while c < nb_images_per_class:
        for string in os.listdir(folder):
            create_new_pictures(folder, string, generator, 10000, 8, (256,256))
        c += 10000

In [11]:
equilibrate_dataset("binary_dataset +imagenet + full frames/test", 10000, my_test_generator)

Found 4753 images belonging to 1 classes.
Found 1344 images belonging to 1 classes.


In [12]:
equilibrate_dataset("binary_dataset +imagenet + full frames/train", 100000,my_generator)

Found 21201 images belonging to 1 classes.
Found 20810 images belonging to 1 classes.
Found 21201 images belonging to 1 classes.
Found 20810 images belonging to 1 classes.
Found 21201 images belonging to 1 classes.
Found 20810 images belonging to 1 classes.
Found 21201 images belonging to 1 classes.
Found 20810 images belonging to 1 classes.
Found 21201 images belonging to 1 classes.
Found 20810 images belonging to 1 classes.
Found 21201 images belonging to 1 classes.
Found 20810 images belonging to 1 classes.
Found 21201 images belonging to 1 classes.
Found 20810 images belonging to 1 classes.
Found 21201 images belonging to 1 classes.
Found 20810 images belonging to 1 classes.
Found 21201 images belonging to 1 classes.


KeyboardInterrupt: 

In [9]:
equilibrate_dataset("boats_id_test", 200 , my_test_generator)

Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging 